## Homework 2
1. Input image from eos_map_x.png.
2. Try to detect the location of the <b>player</b> (in cyan color), and the location of the nearest <b>shrine</b> (in red color). (hint: use <i>inRange()</i>)
3. Try to detect the facing direction of the <b>player</b>. (hint: possibly use <i>morphologyEx()</i>, or <i>findContours(), minEnclosingTriangle()</i> to determine the player axis)
4. Draw a yellow line indicating the facing direction of the <b>player</b>.
5. Draw a yellow line from the <b>player</b> to the <b>shrine</b>.
4. Compute the required <b>rotating angle</b> so the player is facing to the shrine. 
(positive angle means clockwise rotation, negative angle means counterclockwise rotaion) (hint: use <i>atan2()</i>)
5. Print the rotating angle on top-left corner of the output images. (in degree, not radian)
6. Write a simple report in a separate cell.
7. Upload your Jupyter code file (*.ipynb)

In [1]:
import cv2
import numpy as np

def find_max_area(contours):
    areas = [cv2.contourArea(contour) for contour in contours]
    if areas:
        return max(areas)
    else:
        return 0

def find_centers_and_areas(contours, min_area_threshold):
    centers = []
    areas = []
    for contour in contours:
        area = cv2.contourArea(contour)
        areas.append(area)
        if area > min_area_threshold:
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                centers.append((cX, cY))
    return centers, areas

def find_closest_points(centers1, centers2):
    min_dist = float('inf')
    closest_point1 = None
    closest_point2 = None
    for center1 in centers1:
        for center2 in centers2:
            dist = np.linalg.norm(np.array(center1) - np.array(center2))
            if dist < min_dist:
                min_dist = dist
                closest_point1 = center1
                closest_point2 = center2
    return closest_point1, closest_point2

# 读取图像
img = cv2.imread('eos_map_0.png')
img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# 定义颜色区域的HSV范围
red_low = np.array([0, 100, 100])  
red_up  = np.array([0, 255, 255])    
blue_low = np.array([90, 100, 100])
blue_up  = np.array([120, 255, 255])  

# 创建红色和蓝色的遮罩
red_mask  = cv2.inRange(img_hsv, red_low,  red_up)
blue_mask = cv2.inRange(img_hsv, blue_low, blue_up)

# 对遮罩进行形态学操作
kernel_dot = np.ones((3,3), np.uint8)
red_mask = cv2.morphologyEx(red_mask, cv2.MORPH_CLOSE, kernel_dot)
blue_mask = cv2.morphologyEx(blue_mask, cv2.MORPH_CLOSE, kernel_dot)

# 找到红色区域的中心点及面积
red_contours, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
red_centers, red_areas = find_centers_and_areas(red_contours, 110)
max_red_area = find_max_area(red_contours)

# 找到蓝色区域的中心点及面积
blue_contours, _ = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
blue_centers, blue_areas = find_centers_and_areas(blue_contours, 40)
max_blue_area = find_max_area(blue_contours)

# 找到最接近的红色和蓝色中心点
if red_centers and blue_centers:
    closest_red, closest_blue = find_closest_points(red_centers, blue_centers)

# 其余部分保持不变



In [2]:
# 近似藍色區域的輪廓
epsilon = 0.1 * cv2.arcLength(blue_contours[0], True)
approx = cv2.approxPolyDP(blue_contours[0], epsilon, True)

# 找到凸包
hull = cv2.convexHull(approx)

# 提取凸包的頂點
hull_points = [tuple(point[0]) for point in hull]


#intersection = hull_points[2]

# 計算兩條長邊的交點
side_length_ab =  ((hull_points[0][0] - hull_points[1][0])** 2 + (hull_points[0][1] - hull_points[1][1])** 2 ) ** 0.5
side_length_ac =  ((hull_points[0][0] - hull_points[2][0])** 2 + (hull_points[0][1] - hull_points[2][1])** 2 ) ** 0.5
side_length_bc =  ((hull_points[1][0] - hull_points[2][0])** 2 + (hull_points[1][1] - hull_points[2][1])** 2 ) ** 0.5


max_side_length = max(side_length_ab + side_length_ac , side_length_bc + side_length_ac, side_length_ab + side_length_bc)

if max_side_length == side_length_ab + side_length_ac: #a
    intersection = hull_points[0]
    
if max_side_length == side_length_ab + side_length_bc: #b
    intersection = hull_points[1]
    
if max_side_length == side_length_ac + side_length_bc: #c
    intersection = hull_points[2]



vector = np.array(closest_red) - np.array(intersection)

extended_vector = 2 * vector

cv2.line(image, closest_red, intersection, (0, 255, 255), 2)


cv2.line(image, intersection, tuple(extended_vector), (0, 255, 255), 2)


cv2.circle(image, intersection, 5, (255, 0, 0), -1)
img_with_yellow_lines = cv2.imread('path_to_image')
print('red_center:', closest_red)


blue_center = closest_blue
red_center = closest_red
intersection = intersection


vector_blue = np.array(blue_center) - np.array(intersection)
vector_red = np.array(red_center) - np.array(intersection)


cosine_angle = np.dot(vector_blue, vector_red) / (np.linalg.norm(vector_blue) * np.linalg.norm(vector_red))
angle = np.arccos(cosine_angle)
angle = np.degrees(angle)



if vector_blue[1] < 0:
    angle = -angle
if vector_red[1] < 0:
    angle = -angle



print('angle:', angle)



cv2.putText(image, f'angle: {angle:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)



cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

red_center: (117, 86)
angle: -124.08588699418924


[ WARN:0@0.033] global loadsave.cpp:248 findDecoder imread_('path_to_image'): can't open/read file: check file path/integrity
qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/infor/miniconda3/envs/computervision/lib/python3.9/site-packages/cv2/qt/plugins"
